- https://hcnoh.github.io/2018-12-11-tacotron
- https://tv.naver.com/v/2292650
- http://iscslp2018.org/images/T4_Towards%20end-to-end%20speech%20recognition.pdf
- https://google.github.io/tacotron/publications/tacotron/index.html
- https://google.github.io/tacotron/index.html

In [1]:
import torch
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

## Tacotron

2017년도 구글에서 발표된 Tacotron이라는 논문입니다. 이 논문에서 해결하는 Task는 Speech Synthesis 라는 음성합성입니다. 간단하게 말하면 Input으로 텍스트를 넣으면 타코트론이 Spectrogram을 생성하게 도와줍니다. 즉 Data가 <Text,Audio> 페어가 들어온다면, 이 둘 사이의 맵핑을 End-to-End로 학습시켜주는 것입니다.

### Related Works

__WaveNet(2016)__<br>
- 샘플 수준의 Auto-Regressive Model이기 때문이 너무 느리다
- TTS로 바로 활용할 수 없으며, TTS-Frentend로 부터 Lingustic Feature를 입력으로 넣어주어야한다.
- Tacogron의 Vocoder로 사용되었다.

__DeepVoice(2017)__<br>
- TTS 파이프 라인의 파트들을 뉴럴넷 모델로 대체하였다.
- 학습이 End-to-End로 작동하지는 않는다 .

문자열을 입력으로, 음성의 스펙트럼 벡터열을 출력으로 지정하여, 문자열이 스펙트럼으로 변환되는 중간 과정을 자동으로 학습합니다. 이때 입력과 출력의 길이의 차이가 발생합니다. 이때 Attention을 도입하여 입력과 출력 사이의 관계를 학습하게 됩니다.

`Text -> Waveform`

이 각각의 과정을 크게 Encoding, Decoding, Post Processing으로 나눠서 생각할 수 있습니다. 

- Input : Text
- Embedding
- Encoding : Robust Representation
- Decoding : Mel-Spectrogram
- Post-Processing : Linear Sepctrogram and waveform

<img src="./img/tacotron.png">

## Embedding

Embedding과정에서는 Input text가 일반적으로 Character-Level로 Embedding을 거치게 됩니다. 음절이나 단어 단위가 아니라 Character 단위라는것은, 일반적으로 `ㄱ,ㄴ,ㄷ` 처럼 음소단위로 구성될 것이라는 것입니다. 

그 다음 문자 임베딩은 Fully Connected와 Dropout으로 구성된 Pre-Net을 거치게 됩니다. 그 다음은 CBHG라는 모델로 들어가게 됩니다.

- Embedding은 일반적으로 상대적 고차원 의미공간으로 맵핑시키는 것이다. 하지만 `ㄱ,ㄴ,ㄷ` 같은 값을 Embedding한다는 것은 무슨 의미를 가지는 것인가
- Pre-Net에서 Fully Connected Layer가 하는 것은 음소단위의 선형, 비선형 결합 그것이 의미하는 것은?

In [2]:
in_dim = 80
sizes = [256,128]
in_sizes = [in_dim] + sizes[:-1]
nn.ModuleList([nn.Linear(in_size,out_size) for (in_size, out_size) in zip(in_sizes, sizes)])

ModuleList(
  (0): Linear(in_features=80, out_features=256, bias=True)
  (1): Linear(in_features=256, out_features=128, bias=True)
)

In [3]:
class Prenet(nn.Module):
    def __init__(self, in_dim, sizes=[256,128]):
        super(Prenet, self).__init__()
        in_sizes = [in_dim] + sizes[:-1]
        self.layers = nn.ModuleList(
            [nn.Linear(in_size,out_size) for (in_size, out_size) in zip(in_sizes, sizes)])
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, inputs):
        for linear in self.layers:
            inputs = self.dropout(self.relu(linear(inputs)))
        return inputs

### CBHG
2017년 구글이 발표한 TTS 논문인 타코트론에서 사용한 신경망 블록입니다. Convolutional 1D filters, bank, highway networks, gated recurrent unit bidirectional의 첫 글자를 따서 지어진 이름입니다. 단어가 나열된 순서대로 각 블록을 쌓아 만든 구조이기 때문입니다.

<img src="./img/CBHG.png">

__1D Convolution__<br>
1D Convolution의 경우에는 Time step을 축으로 따라서 1D Convolution이 이루어 지는데 이때, Convolution Bank에 대해서는 인코더의 Kernel 사이즈를 1부터 16까지 16개를 사용하게 됩니다. 뒤에도 나오지만 디코더의 PostNet에서 사용되는 CBHG의 경우에는 Kernel사이즈를 1부터 8까지 8개를 사용하게 됩니다. 각각의 Kernel을 이용하여 1D Convolution을 수행한 뒤, 그 결과를 Concat하여 넘기게 됩니다.

Convolution을 거친 후에는 시간축에 따라서 Max Pooling을 진행하게 됩니다. 그 다음에 다시 1D Convolution을 거친 후, 입력 레이어와의 Residual Connection을 만들고, HighWay Layer로 넘어갑니다.

__HighWay Network__<br>
Highway Network는 Gating 구조를 사용하는 Residual네트워크입니다. Resnet으로 익숙한 이 구조는 Vanishing Gradient 문제를 해결하기 위해서 고안된 구조입니다. Residual Network의 Backpropagation을 구하면 Gradient를 입력부분에 직접적으로 전달할수 있는 장점이 있다고 합니다. 

Highway network의 경우에는 이러한 Residual을 조금 다르게 구성하게 됩니다. Gating을 통해서 Residual의 비율을 모델이 학습하게 하는 것입니다. 
$$
\text{Highway}(x) = T(x) * H(x) + (1-T(x)) * x
$$

여기서 $T(x)$를 Transfrom Gate라고 하는데, 이 게이트 function의 Bias는 -1과 같이 음수로 초기화해야합니다. 

__Bidirectional GRU__<br>
RNN계열의 네트워크를 통해서 데이터들을 모두 Sequence의 관계를 학습하게 됩니다. 두개의 방향에서 오게되는 시퀀스들을 Concat하여 출력하게 됩니다.

In [4]:
class BatchNormConv1d(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size, stride, padding, activation = None):
        super(BatchNormConv1d, self).__init__()
        self.conv1d = nn.Conv1d(in_dim, out_dim, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm1d(out_dim)
        self.activation = activation
    
    def forward(self, x):
        x = self.conv1d(x)
        if self.activation is not None:
            x = self.activation(x)
        return self.bn(x)        

In [5]:
class Highway(nn.Module):
    def __init__(self, in_size, out_size):
        super(Highway, self).__init__()
        self.H = nn.Linear(in_size, out_size)
        self.H.bias.data.zero_()
        self.T = nn.Linear(in_size, out_size)
        self.T.bias.data.fill_(-1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, inputs):
        H = self.relu(self.H(self))
        T = self.sigmoid(self.T(inputs))
        return H * T + inputs * (1.0 - T)

In [6]:
[Highway(in_dim,in_dim) for __ in range(4)]

[Highway(
   (H): Linear(in_features=80, out_features=80, bias=True)
   (T): Linear(in_features=80, out_features=80, bias=True)
   (relu): ReLU()
   (sigmoid): Sigmoid()
 ), Highway(
   (H): Linear(in_features=80, out_features=80, bias=True)
   (T): Linear(in_features=80, out_features=80, bias=True)
   (relu): ReLU()
   (sigmoid): Sigmoid()
 ), Highway(
   (H): Linear(in_features=80, out_features=80, bias=True)
   (T): Linear(in_features=80, out_features=80, bias=True)
   (relu): ReLU()
   (sigmoid): Sigmoid()
 ), Highway(
   (H): Linear(in_features=80, out_features=80, bias=True)
   (T): Linear(in_features=80, out_features=80, bias=True)
   (relu): ReLU()
   (sigmoid): Sigmoid()
 )]

In [7]:
class CBHG(nn.Module):
    def __init__(self, in_dim, K=16, projections=[128,128]):
        super(CBHG, self).__init__()
        self.in_dim = in_dim
        self.relu = nn.ReLU
        self.conv1d_banks = nn.ModuleList(
            [BatchNormConv1d(in_dim, in_dim, kernel_size=k, strid=1, padding=k//2, activation=self.relu)
                for k in range(1, K+1)])
        self.maxpool1d = nn.MaxPool1d(kernel_size=2, stride=1, padding=1)
        
        # setting
        in_size = [K*in_dim] + projections[:-1]
        activations = [self.relu] * [len(projections) - 1] + [None]
        
        
        # Main layers
        self.conv1d_projections = nn.Module(
            [BatchNormConv1d(in_size, out_size, kernel_size=3, stride=1, padding=1, activation=ac)
                for (in_size, out_size, ac) in zip(in_sizes, projections, activations)])
        self.pre_highway = nn.Linear(projections[-1], in_dim, bias=False)
        self.highway = nn.Module(
            [Highway(in_dim,in_dim) for __ in range(4)])
        
        self.gru = nn.GRU(
            in_dim, in_dim, 1, batch_first = True, bidirectional = True)
        
    def forward(self, inputs, input_lengths = None):
        # (B, T_in, in_dim)
        x = inputs
        
        # Convolution for Time axis
        if x.size(-1) == self.in_dim:
            x = x.transpose(1,2)
        T = x.size(-1)
        
        # Concat convd bank outputs
        x = torch.cat([conv1d(x)[:,:,:T] for conv1d in self.conv1d_banks], dim=1)
        assert x.size(1 ) == self.in_dim * len(self.conv1d_banks)
        x = self.max_pool1d(x)[:,:,:T]
        
        for conv1d in self.conv1d_projections:
            x = conv1d(x)
            
        # Back to original shape
        # (B, T_in, in_dim)
        x = x.transpose(1,2)
        
        if x.size(-1) != self.in_dim:
            x = self.pre_highway
            
        # Residual Connection
        x += inputs
        for high
        

SyntaxError: invalid syntax (<ipython-input-7-f39c94197d42>, line 53)

### Attention


In [ ]:
class BahdanauAttention(nn.Module):
    def __init__(self, dim):
        super(BahdanauAttention, self).__init__()
        self.query_layer = nn.Linear(dim, dim, bias=False)
        self.tanh = nn.Tanh()
        self.v = nn.Linear(dim,1, bias=Fasle)
    
    def forward(self, query, processed_memory):
        """
        Args:
            query: (batch, 1, dim)
            processed_memory : (batch, max_time, dim)
        """
        if query.dim() == 2:
            # insert time-axis for broadcasting
            query = query,unsqueeze(1)
        # (batch, 1, dim)
        processed_query = self.query_layer(query)
        
        # (batch, max_time, 1)
        alignment = self.v(self.tanh(processed_query + processed_memory))
        
        # (batch, max_time)
        return alignment.squeeze(-1)
    
def get_mask_from_lengths(memory, memory_lengths):
    mask = memory.data.new(memory.size(0), memory.size(1)).byte().zero_()
    for idx, l in enumerate(memory_lengths):
        mask[idx][:l] = 1
    return ~mask

### Decoding

디코더의 경우에는 인코더와 마찬가지로, Pre-net을 지난 후에 Attention RNN과 Decoder RNN 셀을 지나게 됩니다. Attention RNN셀은 GRU셀로 구성이 되고, Decoder RNN셀은 Residual GRU Cell이 두 층으로 되어 있습니다.

디코더의 결과는 Mel-Spectrogram을 출력하게 되는데, 여기서 Reduction Factor $r$을 이용하여 여러 프레임을 동시에 생성한다는 것입니다. 만약 Reduction Factor가 3으로 설정 되어 있다면, 디코더는 한 타임스텝에서 3개의 프레임에 해당하는 Mel-Spectrogram을 생성한다는 것입니다. 이는 음성 신호의 연속성을 반영한 것입니다. 

### Post Processing

## Tranining Method

Tacotron은 Encoder-Decoder 구조이기 때문에, Tranining 과 Generative 방식 역시 Encdoer-Decoder 구조를 따르게 됩니다. 트레이닝의 경우에는 입력 텍스트와 오디오 페어가 존재해야한다. 더 정확히는 전처리가 완료된 Mel-Spectrogram과 Linear-Spectrogram을 준비해야합니다. <Text,'Mel-Spectrogram,Linear-Spectrogram'>이 있어야한다.

### Loss function
Loss 구성은 다음과 같다. 먼저 디코더의 출력인 Mel-Spectrogram출력과 타겟 Mel-Spectrogram과의 L1 Distance를 구해야한다. 이를 mel_loss라고 정의하도록하자. 마찬가지 방법으로 Linear-Spectrogram 출력과 타겟 Linear-Spectrogram과의 L1 Distance를 구한다. 이를 lin_loss라고 불르다. 최종적인 타코트론의 Loss는 Mel-Loss 및 Linear-Loss의 합으로 정의된다.

### Inference


<img src="./img/params.png">